In [1]:
# Import all packages
import numpy as np
import pandas as pd

In [2]:
# Import all datasets
df_employees = pd.read_excel("/data")
df_employee_turnover = pd.read_excel("/data") 
df_event_report = pd.read_excel ("/data")
df_terminated_employments = pd.read_excel ("/data")
df_perf_2017 = pd.read_csv ("/data")
df_perf_2018 = pd.read_csv ("/data")
df_perf_2019 = pd.read_csv ("/data")
df_perf_2020 = pd.read_csv ("/data")
df_perf_2021 = pd.read_csv ("/data")
df_perf_2022 = pd.read_csv ("/data")
df_perf_2023 = pd.read_csv ("/data")

In [3]:
# Observe only office and field employees from the event report
filtered_event_report = df_event_report[df_event_report['Employee Type'].isin(['Office & Field'])]

In [4]:
# Observe only office and field employees from the emplyees_turnover_promotions report
filtered_employee_turnover = df_employee_turnover[df_employee_turnover['c'].isin(['Office & Field'])]

In [5]:
merged_employee_events = filtered_event_report.merge (filtered_employee_turnover, on = "User/Employee ID", how = "left")

In [ ]:
merged_employee_events.head()


In [ ]:
merged_employee_events = merged_employee_events.rename(columns={'User/Employee ID': 'Username'})
merged_employee_events.head()

In [8]:
# Observe only office and field employees from the event report
filtered_employee_data = df_employees[df_employees['Employee Type'].isin(['Office & Field'])]

In [ ]:
filtered_employee_data.head()

In [10]:
all_employee_data = filtered_employee_data.merge(merged_employee_events, on = "Username", how = "left")

In [ ]:
all_employee_data.head()
all_employee_data.info()
all_employee_data.shape

In [13]:
employees_and_terminations = all_employee_data.merge(df_terminated_employments[['Employee ID', 'Termination Date', 'Event (Label)', 'Event Reason Icode (Event Name)', 'Regret Termination']], on='Employee ID', how='left', suffixes = ('', '_new'))

In [15]:
employees_and_terminations['Employment Details Termination Date'] = employees_and_terminations['Employment Details Termination Date'].combine_first(employees_and_terminations['Employment Details Termination Date_x']).combine_first(employees_and_terminations['Employment Details Termination Date_y']).combine_first(employees_and_terminations['Termination Date'])

In [17]:
def update_employee_status(df):
    # Check if 'Employment Details Termination Date' is not null
    condition = df['Employment Details Termination Date'].notna()
    # Update 'Employee Status' to 'Terminated' where the condition is True
    df.loc[condition, 'Employee Status'] = 'Terminated'
    return df

In [18]:
employees_and_terminations = update_employee_status(employees_and_terminations)

In [19]:
# Check the distribution of the employee status
class_counts_employee_status = employees_and_terminations['Employee Status'].value_counts()
print(class_counts_employee_status)

Active          5989
Terminated      3403
Unpaid Leave     224
Dormant           19
Name: Employee Status, dtype: int64


In [20]:
# Before merging all performance ratings reports into one, add a 'Year' column to every single one of them and fill in the corresponding year of the annual evalutation
df_perf_2017['Year'] = 2017
df_perf_2018['Year'] = 2018
df_perf_2019['Year'] = 2019
df_perf_2020['Year'] = 2020
df_perf_2021['Year'] = 2021
df_perf_2022['Year'] = 2022
df_perf_2023['Year'] = 2023

In [21]:
# Standardizing all columns before appending
df_perf_2017 = df_perf_2017.rename(columns={'Subject User ID': 'Subject User Name'})
df_perf_2021.drop('Subject Pay Grade', axis=1)
df_perf_2022.drop(columns = ['Subject Pay Grade','ELT-1'] , axis=1, inplace = True)
df_perf_2023.drop(columns = ['Subject Pay Grade','ELT-1'] , axis=1, inplace = True)

In [22]:
# Append all performance datasets together
total_perf_eval = pd.concat([df_perf_2017, df_perf_2018, df_perf_2019, df_perf_2020, df_perf_2021, df_perf_2022, df_perf_2023], ignore_index=True)

In [ ]:
total_perf_eval.info

In [24]:
# Pivoting data to wide format using pivot_table
# Creating a multi-level column index for each type of performance rating by year
wide_total_perf_eval = total_perf_eval.pivot_table(
    index='Subject User Name', 
    columns='Year', 
    values=['Overall Performance Rating', 'Overall Objective Rating', 'Overall Competency Rating'],
    aggfunc='first'  # Using 'first' to take the first entry if there are duplicates
)

# Optionally, fill missing values, e.g., forward fill
wide_total_perf_eval.fillna(method='ffill', inplace=True)

# Rename 'Subject User Name' into 'Username'
wide_total_perf_eval.index.rename('Username', inplace=True)

In [ ]:
wide_total_perf_eval.head()
wide_total_perf_eval.shape

In [26]:
# Flatten the columns with a custom format
wide_total_perf_eval.columns = [' '.join([str(col) for col in cols]) for cols in wide_total_perf_eval.columns.values]

In [27]:
# Merge the flattened 'wide_total_perf_eval' with 'employees_and_terminations'
final_df = pd.merge(employees_and_terminations, wide_total_perf_eval, on='Username', how='left')

In [ ]:
final_df['Employment Details Termination Date'].count()

In [29]:
final_df = update_employee_status(final_df)

In [ ]:
final_df.to_csv('UPDATED_Employee turnover_combined dataset.csv')